# Whatsapp Web Automation

**Author: Jimmy Lam**

**_Created: 2022-04-25_**

v1.0: Public Release

**Part 1 - Import libraries and launch Whatsapp Web**

In [ ]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
import time

chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://web.whatsapp.com/')

#wait 60 secs to allow for the user to manually scan the Whatsapp Web QR code to log on
el_side = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.ID, "side")))

#locate the search box
el_search = el_side.find_element(By.XPATH, "//div[contains(@title, 'Search')]")
print("Logged in and located search box:", el_search)

**Part 2 - Set up list(s) of chat groups**

In [ ]:
list_chat_groups_test = [
    'TestGroup01',
    'TestGroup02',
    'TestGroup03',
    'TestGroup04',
    'TestGroup05',
    'TestGroup06',
    'TestGroup07',
    'TestGroup08',
    'TestGroup09',
    'TestGroup10'
]

list_chat_groups_2 = [
    #insert your own list here
]

**Part 3 - Define the automation functions**

_click_modal_button_ : a helper function used by the other functions.

_add_contact_to_group(group_name, contact_to_add)_ : adds **contact_to_add** to **group_name**

_remove_contact_from_group(group_name, contact_to_remove)_ : removes **contact_to_remove** from **group_name**

_make_group_admin(group_name, contact_to_add)_ : sets **contact_to_add** as admin of **group_name**

_dismiss_as_group_admin(group_name, contact_to_remove)_ : dismisses **contact_to_remove** as admin of **group_name**

In [ ]:
#define a helper function
def click_modal_button(button_text):    
    modal_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-animate-modal-body='true']//div[@role='button']//div[text() = '%s']" % (button_text))))
    modal_button.click()                                                      

#define a function that adds contact_to_add to group_name
def add_contact_to_group(group_name, contact_to_add):
    #find chat with the correct title
    el_target_chat = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//span[@title='%s']" % (group_name))))
    el_target_chat.click()
        
    #wait for it to load by detecting that the header changed with the new title
    el_header_title = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='main']//header//span[@title='%s']" % (group_name))))
    
    #click on the menu button
    el_menu_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='main']//div[@data-testid='conversation-menu-button']")))
    el_menu_button.click()
    
    #click on the Group Info button
    el_group_info = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='app']//li//div[@aria-label='Group info']")))
    el_group_info.click()    
    
    #click on the Add Participant button
    el_add_participant = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='drawer-right']//div[text() = 'Add participant']")))
    el_add_participant.click()    
    
    #click on the Search
    el_modal_popup = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-animate-modal-body='true']")))    
    el_modal_popup.find_element(By.XPATH, "//div[contains(@title, 'Search')]").send_keys(contact_to_add)
    
    #click on the Contact
    el_contact_to_add = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-animate-modal-body='true']//span[@title='%s']" % (contact_to_add))))
    el_contact_to_add.click()    
    
    #check whether already added
    if len(el_modal_popup.find_elements(By.XPATH, "//div[text() = 'Already added to group']")) > 0:
        print(contact_to_add + ' was already an existing participant of ' + group_name)
        el_modal_popup.find_element(By.XPATH, "//header//button[@aria-label='Close']").click()
    else:    
        #click on the Green Check Mark
        el_green_check = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-animate-modal-body='true']//div[@role='button']//span[@data-testid='checkmark-medium']")))
        el_green_check.click()        

        #click on the Add Participant
        click_modal_button('Add participant')
        print(contact_to_add + ' added to ' + group_name)


def remove_contact_from_group(group_name, contact_to_remove):
    #find chat with the correct title
    el_target_chat = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//span[@title='%s']" % (group_name))))
    el_target_chat.click()    
        
    #wait for it to load by detecting header changed
    el_header_title = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='main']//header//span[@title='%s']" % (group_name))))
        
    #click on the menu button
    el_menu_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='main']//div[@data-testid='conversation-menu-button']")))
    el_menu_button.click()
    
    #click on the Group Info button
    el_group_info = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='app']//li//div[@aria-label='Group info']")))
    el_group_info.click()
    
    #wait until target user can be found
    try:
        el_target_contact = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='drawer-right']//div[@role='gridcell']//span[@title='%s']" % (contact_to_remove))))
        
        #Need to Hover over Contact
        #Ref: https://www.roelpeters.be/mouseover-in-selenium-hover/            
        ActionChains(driver).move_to_element(el_target_contact).perform() #hover over

        #Wait for dropdown arrow to appear, then click it.
        #Ref: https://stackoverflow.com/questions/27934945/selenium-move-to-element-does-not-always-mouse-hover
        el_dropdown_arrow = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='drawer-right']//div[@role='gridcell']//button[@aria-label='Open the chat context menu']")))
        el_dropdown_arrow.click()

        #click on the Make group admin button
        el_remove_from_group = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='app']//li//div[@aria-label='Remove']")))
        el_remove_from_group.click()

        #click on the green button
        click_modal_button('Remove')
        print(contact_to_add + ' removed from ' + group_name)
    except:
        print("Error occured while finding user in chat")
        
        
def make_group_admin(group_name, contact_to_add):
    #find chat with the correct title
    el_target_chat = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//span[@title='%s']" % (group_name))))
    el_target_chat.click()    
        
    #wait for it to load by detecting header changed
    el_header_title = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='main']//header//span[@title='%s']" % (group_name))))
        
    #click on the menu button
    el_menu_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='main']//div[@data-testid='conversation-menu-button']")))
    el_menu_button.click()
    
    #click on the Group Info button
    el_group_info = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='app']//li//div[@aria-label='Group info']")))
    el_group_info.click()
    
    #wait until target user can be found
    el_target_contact = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='drawer-right']//div[@role='gridcell']//span[@title='%s']" % (contact_to_add))))
    
    #check whether user is already admin by looking for the Group admin label
    has_group_admin_label = len(el_target_contact.find_elements(By.XPATH, "../../..//div[text() = 'Group admin']"))    
    if has_group_admin_label > 0:    
        print(contact_to_add + (' were' if contact_to_add == 'You' else ' was') + ' already an existing admin of ' + group_name)
    else:
        #Need to Hover over Contact
        #Ref: https://www.roelpeters.be/mouseover-in-selenium-hover/            
        ActionChains(driver).move_to_element(el_target_contact).perform() #hover over

        #Wait for dropdown arrow to appear, then click it.
        #Ref: https://stackoverflow.com/questions/27934945/selenium-move-to-element-does-not-always-mouse-hover
        el_dropdown_arrow = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='drawer-right']//div[@role='gridcell']//button[@aria-label='Open the chat context menu']")))
        el_dropdown_arrow.click()
    
        #click on the Make group admin button
        el_make_group_admin = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='app']//li//div[@aria-label='Make group admin']")))
        el_make_group_admin.click()
    
        #click on the green button
        click_modal_button('Make group admin')
        print(contact_to_add + ' added as admin of ' + group_name)

def dismiss_as_group_admin(group_name, contact_to_remove):
    #find chat with the correct title
    el_target_chat = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//span[@title='%s']" % (group_name))))
    el_target_chat.click()    
        
    #wait for it to load by detecting header changed
    el_header_title = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='main']//header//span[@title='%s']" % (group_name))))
        
    #click on the menu button
    el_menu_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='main']//div[@data-testid='conversation-menu-button']")))
    el_menu_button.click()
    
    #click on the Group Info button
    el_group_info = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='app']//li//div[@aria-label='Group info']")))
    el_group_info.click()
    
    #wait until target user can be found
    el_target_contact = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='drawer-right']//div[@role='gridcell']//span[@title='%s']" % (contact_to_remove))))
    
    #Check whether user is already admin by looking for the Group admin label
    has_group_admin_label = len(el_target_contact.find_elements(By.XPATH, "../../..//div[text() = 'Group admin']"))    
    if has_group_admin_label < 1:    
        print(contact_to_remove + (' were' if contact_to_remove == 'You' else ' was') + ' not an existing admin of ' + group_name)
    else:
        #Need to Hover over Contact
        #Ref: https://www.roelpeters.be/mouseover-in-selenium-hover/            
        ActionChains(driver).move_to_element(el_target_contact).perform() #hover over

        #Wait for dropdown arrow to appear, then click it.
        #Ref: https://stackoverflow.com/questions/27934945/selenium-move-to-element-does-not-always-mouse-hover
        el_dropdown_arrow = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='drawer-right']//div[@role='gridcell']//button[@aria-label='Open the chat context menu']")))
        el_dropdown_arrow.click()
    
        #click on the Make group admin button
        el_make_group_admin = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='app']//li//div[@aria-label='Dismiss as admin']")))
        el_make_group_admin.click()
            
        print(contact_to_remove + ' removed as admin of ' + group_name)                                   


In [ ]:
start_time = datetime.now()

#loop through a provided list of chat groups to perform an action
#REPLACE list_chat_groups_test with your chosen list name defined in Part 2
for chat_name in list_chat_groups_test:
        
    el_search.clear()
    el_search.send_keys(chat_name)
    
    try:

        ##########################################################
        ## COMMENT OUT the #1, #2, #3 or #4 BLOCKS as necessary ##
        ##########################################################
        
        #1 Add Contact to Group
        print('Attempting to add to', chat_name, ":")
        add_contact_to_group(chat_name, 'INSERT USER NAME HERE')

#         #2 Remove Contact from Group
#         print('Attempting to remove from group ', chat_name, ":")
#         remove_contact_from_group(chat_name, 'INSERT USER NAME HERE')      
        
#         #3 Set Contact as Group Admin
#         print('Attempting to make group admin of ', chat_name, ":")
#         make_group_admin(chat_name, 'INSERT USER NAME HERE')

#         #4 Dismiss Contact as Group Admin
#         print('Attempting to dismiss as admin of ', chat_name, ":")
#         dismiss_as_group_admin(chat_name, 'INSERT USER NAME HERE')

        
    except Exception as exception:
        print("Exception: {}".format(type(exception).__name__))
        print("Exception message: {}".format(exception))
    
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))